In [34]:
import os
import sagemaker
import numpy as np
import tensorflow as tf
from sagemaker.tensorflow import TensorFlow

In [53]:
ON_SAGEMAKER_NOTEBOOK = True

# train_instance_type='local_gpu'
train_instance_type='ml.p2.xlarge'
deploy_instance_type='ml.c5.2xlarge'

bucket = 'sagemaker-mumbai-data-set'
key = 'ena_data'

In [36]:
train_input_path = "s3://{}/{}/train/".format(bucket, key)
validation_input_path = "s3://{}/{}/validation/".format(bucket, key)

In [37]:
sagemaker_session = sagemaker.Session()
if ON_SAGEMAKER_NOTEBOOK:
    role = sagemaker.get_execution_role()
else:
    role = '[IAM ROLE]'

In [38]:
hyperparameters = {'epochs': 50, 'batch_size' : 64}

In [30]:
os.path.abspath('')

'/home/ec2-user/SageMaker/sagemaker_imageclassify/script_mode'

In [31]:
estimator = TensorFlow(base_job_name='ena-classify-mobilenet-1',
#                        entry_point=os.path.join(os.path.abspath(''),'entry.py'),
                       entry_point='entry.py',
                       source_dir='src',
                       model_dir='s3://quikr-datascience-mumbai/image_classify/ena_classify/sagemaker_output/',
                       role=role,
                       framework_version='2.0.0',
                       py_version='py3',
                       output_path='s3://quikr-datascience-mumbai/image_classify/ena_classify/sagemaker_output/',
                       hyperparameters=hyperparameters,
                       train_instance_count=1, train_instance_type=train_instance_type)

In [ ]:
print("Training ...")
local_inputs = {'train' : train_input_path, 
                'validation' : validation_input_path}
estimator.fit(inputs=local_inputs)

Training ...
2020-02-20 02:56:54 Starting - Starting the training job...
2020-02-20 02:57:14 Starting - Launching requested ML instances......
2020-02-20 02:58:08 Starting - Preparing the instances for training......
2020-02-20 02:59:02 Downloading - Downloading input data.........
2020-02-20 03:00:48 Training - Training image download completed. Training in progress..2020-02-20 03:00:52,915 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-02-20 03:00:53,272 sagemaker-containers INFO     Installing module with the following command:
/usr/bin/python3 -m pip install . -r requirements.txt
Processing /opt/ml/code
  Created wheel for sagemaker-image-classify: filename=sagemaker_image_classify-1.0-cp36-none-any.whl size=1191 sha256=2591cdee5a6ae621889ae46622a768985385d16aaafe8d28b5ae46c7232112c3
  Stored in directory: /tmp/pip-ephem-wheel-cache-nxezyzkw/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
Successfully built sagemaker

getting data
Found 12600 images belonging to 14 classes.
Found 1400 images belonging to 14 classes.
#015   8192/9406464 [..............................] - ETA: 3:26#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  40960/9406464 [..............................] - ETA: 1:22#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 106496/9406464 [..............................] - ETA: 47s #010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010

#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 96/197 [=============>................] - ETA: 43s - loss: 7.3130 - accuracy: 0.6279#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 97/197 [=============>................] - ETA: 43s - loss: 7.2807 - accuracy: 0.6282#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015192/197 [============================>.] - ETA: 2s - loss: 5.1111 - accuracy: 0.6695#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015193/197 [============================>.] - ETA: 1s - loss: 5.0951 - accuracy: 0.6699#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010

.6072 - accuracy: 0.7572#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 97/197 [=============>................] - ETA: 37s - loss: 1.6061 - accuracy: 0.7571#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 98/197 [=============>................] - ETA: 37s - loss: 1.6038 - accuracy: 0.7569#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

...

racy: 0.8011#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015193/197 [============================>.] - ETA: 1s - loss: 0.9481 - accuracy: 0.8014#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015194/197 [============================>.] - ETA: 1s - loss: 0.9487 - accuracy: 0.8010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010

.8796 - accuracy: 0.8211#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 97/197 [=============>................] - ETA: 37s - loss: 0.8818 - accuracy: 0.8206#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 98/197 [=============>................] - ETA: 37s - loss: 0.8825 - accuracy: 0.8205#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

curacy: 0.8051#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015193/197 [============================>.] - ETA: 1s - loss: 0.9312 - accuracy: 0.8048#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015194/197 [============================>.] - ETA: 1s - loss: 0.9324 - accuracy: 0.8042#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

.8778 - accuracy: 0.8170#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 97/197 [=============>................] - ETA: 37s - loss: 0.8771 - accuracy: 0.8174#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 98/197 [=============>................] - ETA: 37s - loss: 0.8770 - accuracy: 0.8175#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

curacy: 0.8027#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015193/197 [============================>.] - ETA: 1s - loss: 0.9240 - accuracy: 0.8025#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015194/197 [============================>.] - ETA: 1s - loss: 0.9246 - accuracy: 0.8021#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

## Deploying

In [49]:
from sagemaker.tensorflow.serving import Model

In [50]:
!ls

src  train.ipynb


In [55]:
model = Model(entry_point='inference.py',
#               dependencies=['requirements.txt'],
              source_dir='./src',
              model_data='s3://quikr-datascience-mumbai/image_classify/ena_classify/sagemaker_output/ena-classify-mobilenet-1-2020-02-20-02-56-53-982/output/model.tar.gz',
              framework_version='2.0.0', role=role)
predictor = model.deploy(initial_instance_count=1, endpoint_name='quikr-eye-v1', instance_type=deploy_instance_type)

---------!